# Лекция 4: Невронни мрежи

## От линейни модели до дълбоко обучение: Neurons, Layers, Backpropagation

**Продължителност:** 2-2.5 часа  
**Предпоставки:** Лекция 3 (Токенизация), основи на калкулус и линейна алгебра  
**Следваща лекция:** Трансформатори (Attention, BERT)

---
## Цели на лекцията

След тази лекция ще можете:

- Обяснявате защо невронните мрежи надграждат линейните модели
- Изграждате feedforward мрежа от нулата с NumPy
- Прилагате алгоритъма за обратно разпространение на сигнала (backpropagation)
- Сравнявате оптимизатори: SGD, Momentum, Adam
- Разчитате training curves и идентифицирате пренагаждане
- Прилагате невронна мрежа за текстова класификация

### Пътна карта за днес

```
Мотивация → Неврони → Forward Pass → Loss → Backprop → Оптимизатори → Training → NLP
```

In [ ]:
# Основни библиотеки
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
# Sklearn за данни и baseline модели
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Настройки за визуализации
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

# Фиксираме random seed за възпроизводимост
np.random.seed(42)

print("✓ Всички библиотеки са заредени успешно!")

---
## 1. Мотивация: От линейни модели към невронни мрежи

### Какво научихме досега?

| Лекция | Концепция | Ограничение |
|--------|-----------|-------------|
| **1** | Logistic Regression + BoW | Линеен модел, няма взаимодействия |
| **2** | Word2Vec embeddings | Статични вектори, без контекст |
| **3** | Tokenization (BPE) | Подготвя входа, но не моделира |

### Защо линейните модели не са достатъчни?

**Линеен модел:** $\hat{y} = \sigma(\mathbf{w}^T \mathbf{x} + b)$

**Проблем:** Може само да раздели данните с хиперравнина.

Не може да улови:
- **Взаимодействия:** "not good" ≠ "not" + "good"
- **Йерархия:** ниско ниво (букви) → средно (думи) → високо (теми)
- **Нелинейни граници:** XOR проблем

In [ ]:
# Демонстрация: XOR проблем - линейно неразделим
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])  # XOR: различни = 1, еднакви = 0

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Ляво: XOR данни
colors = ['red' if y == 0 else 'blue' for y in y_xor]
axes[0].scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=200, edgecolors='black', linewidth=2)
axes[0].set_xlabel('$x_1$')
axes[0].set_ylabel('$x_2$')
axes[0].set_title('XOR: Може ли една права да раздели цветовете?')
axes[0].set_xlim(-0.5, 1.5)
axes[0].set_ylim(-0.5, 1.5)

# Примерни (грешни) линейни граници
x_line = np.linspace(-0.5, 1.5, 100)
for slope, intercept, ls in [(-1, 0.5, '--'), (1, -0.5, ':'), (-1, 1.5, '-.')]:
    axes[0].plot(x_line, slope * x_line + intercept, ls, alpha=0.5, color='gray')

axes[0].legend(['Опит 1', 'Опит 2', 'Опит 3'], loc='upper right')

# Дясно: Решение с невронна мрежа
axes[1].scatter(X_xor[:, 0], X_xor[:, 1], c=colors, s=200, edgecolors='black', linewidth=2)
axes[1].set_xlabel('$x_1$')
axes[1].set_ylabel('$x_2$')
axes[1].set_title('Невронна мрежа: нелинейна граница')
axes[1].set_xlim(-0.5, 1.5)
axes[1].set_ylim(-0.5, 1.5)

# Рисуваме нелинейна граница (ръчно)
xx, yy = np.meshgrid(np.linspace(-0.5, 1.5, 100), np.linspace(-0.5, 1.5, 100))
Z = np.logical_xor(xx > 0.5, yy > 0.5).astype(float)
axes[1].contourf(xx, yy, Z, levels=[0, 0.5, 1], colors=['#ffcccc', '#ccccff'], alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 XOR е линейно неразделим. Невронната мрежа може да създаде нелинейна граница!")

### Какво предлагат невронните мрежи?

| Характеристика | Линеен модел | Невронна мрежа |
|----------------|--------------|----------------|
| **Граница на решение** | Хиперравнина | Произволна форма |
| **Признаци** | Ръчно създадени | Автоматично научени |
| **Композиция** | $f(x) = Wx$ | $f(x) = f_n(f_{n-1}(...f_1(x)))$ |
| **Изразителност** | Ограничена | Универсален апроксиматор |

### Пътят към LLM

```
Невронни мрежи (днес)
    ↓
Трансформатори (Лекция 5)
    ↓
Предварително обучение (Лекция 6)
    ↓
GPT, Claude, LLaMA (Лекции 7-12)
```

**Важно:** Всичко, което научим днес (forward pass, backprop, Adam), се използва и в най-големите LLM!

---
## 2. Градивни елементи: Неврони и слоеве

### Какво е неврон?

**Неврон** = претеглена сума + нелинейност

$$z = \sum_{i=1}^{n} w_i x_i + b = \mathbf{w}^T \mathbf{x} + b$$
$$a = f(z)$$

Където:
- $\mathbf{x}$ — вход (вектор)
- $\mathbf{w}$ — тегла (weights)
- $b$ — bias
- $f$ — активационна функция

In [ ]:
# Прост неврон
def neuron(x, w, b, activation='relu'):
    """Един неврон: z = w·x + b, a = f(z)"""
    z = np.dot(w, x) + b
    
    if activation == 'relu':
        a = np.maximum(0, z)
    elif activation == 'sigmoid':
        a = 1 / (1 + np.exp(-z))
    else:
        a = z  # linear
    
    return z, a

# Пример
x = np.array([1.0, 2.0, 3.0])  # 3 входа
w = np.array([0.5, -0.3, 0.8])  # 3 тегла
b = 0.1  # bias

z, a = neuron(x, w, b, activation='relu')

print("🧠 Един неврон:\n")
print(f"   Вход x = {x}")
print(f"   Тегла w = {w}")
print(f"   Bias b = {b}")
print(f"\n   z = w·x + b = {w[0]}×{x[0]} + {w[1]}×{x[1]} + {w[2]}×{x[2]} + {b}")
print(f"   z = {z:.2f}")
print(f"   a = ReLU(z) = max(0, {z:.2f}) = {a:.2f}")

### Активационни функции

**Защо са нужни?** Без нелинейност, много слоеве = един линеен слой.

$$f(W_2 \cdot f(W_1 \cdot x)) = f(W_2 W_1 x) = f(W x)$$

| Функция | Формула | Диапазон | Използва се |
|---------|---------|----------|-------------|
| **Sigmoid** | $\frac{1}{1+e^{-z}}$ | (0, 1) | Изход за binary classification |
| **Tanh** | $\frac{e^z - e^{-z}}{e^z + e^{-z}}$ | (-1, 1) | По-добра от sigmoid (центрирана) |
| **ReLU** | $\max(0, z)$ | [0, ∞) | **Стандарт** в скритите слоеве |
| **GELU** | $z \cdot \Phi(z)$ | (-0.17, ∞) | GPT, BERT |

In [ ]:
# Визуализация на активационни функции
z = np.linspace(-5, 5, 200)

# Дефиниции
sigmoid = 1 / (1 + np.exp(-z))
tanh = np.tanh(z)
relu = np.maximum(0, z)
leaky_relu = np.where(z > 0, z, 0.01 * z)

# GELU апроксимация
gelu = 0.5 * z * (1 + np.tanh(np.sqrt(2/np.pi) * (z + 0.044715 * z**3)))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Функции
axes[0].plot(z, sigmoid, label='Sigmoid', linewidth=2)
axes[0].plot(z, tanh, label='Tanh', linewidth=2)
axes[0].plot(z, relu, label='ReLU', linewidth=2)
axes[0].plot(z, gelu, label='GELU (GPT)', linewidth=2, linestyle='--')
axes[0].axhline(y=0, color='gray', linewidth=0.5)
axes[0].axvline(x=0, color='gray', linewidth=0.5)
axes[0].set_xlabel('z')
axes[0].set_ylabel('f(z)')
axes[0].set_title('Активационни функции')
axes[0].legend()
axes[0].set_ylim(-1.5, 3)

# Производни (важни за backprop)
sigmoid_deriv = sigmoid * (1 - sigmoid)
tanh_deriv = 1 - tanh**2
relu_deriv = (z > 0).astype(float)

axes[1].plot(z, sigmoid_deriv, label="Sigmoid'", linewidth=2)
axes[1].plot(z, tanh_deriv, label="Tanh'", linewidth=2)
axes[1].plot(z, relu_deriv, label="ReLU'", linewidth=2)
axes[1].axhline(y=0, color='gray', linewidth=0.5)
axes[1].axvline(x=0, color='gray', linewidth=0.5)
axes[1].set_xlabel('z')
axes[1].set_ylabel("f'(z)")
axes[1].set_title('Производни (важни за backpropagation)')
axes[1].legend()
axes[1].set_ylim(-0.1, 1.1)

plt.tight_layout()
plt.show()

print("💡 Наблюдения:")
print("   • Sigmoid/Tanh: производната → 0 за |z| > 3 (vanishing gradient!)")
print("   • ReLU: производната = 1 за z > 0 (градиентът се запазва)")

In [ ]:
# Демонстрация: Vanishing gradient problem
print("⚠️ Vanishing Gradient Problem:\n")

# Sigmoid при големи стойности
z_values = [0, 2, 5, 10]

print(f"{'z':>6} | {'sigmoid(z)':>12} | {'sigmoid\'(z)':>12}")
print("-" * 36)

for z_val in z_values:
    sig = 1 / (1 + np.exp(-z_val))
    sig_deriv = sig * (1 - sig)
    print(f"{z_val:>6} | {sig:>12.6f} | {sig_deriv:>12.6f}")

print("\n💡 При z=10, производната е ~0.00005. Градиентът изчезва!")
print("   ReLU решава този проблем: f'(z) = 1 за z > 0.")

### Слоеве и архитектура

**Feedforward мрежа** (многослоен перцептрон, MLP):

```
Вход x       Скрит слой 1      Скрит слой 2      Изход
[x₁]           [h₁]              [h₁']            [ŷ₁]
[x₂]    →      [h₂]      →       [h₂']     →     [ŷ₂]
[x₃]           [h₃]              [h₃']
 ⋮              ⋮                 ⋮
```

**Нотация:**
- $W^{[l]}$ — тегла на слой $l$, размер $(n_l, n_{l-1})$
- $b^{[l]}$ — bias на слой $l$, размер $(n_l,)$
- $a^{[l]}$ — активации на слой $l$

In [ ]:
# Визуализация на невронна мрежа
def draw_neural_network(ax, layer_sizes, layer_names=None):
    """Рисува невронна мрежа."""
    n_layers = len(layer_sizes)
    max_neurons = max(layer_sizes)
    
    v_spacing = 1.0
    h_spacing = 2.0
    
    # Позиции на невроните
    positions = []
    for i, n_neurons in enumerate(layer_sizes):
        layer_pos = []
        start_y = (max_neurons - n_neurons) / 2 * v_spacing
        for j in range(n_neurons):
            x = i * h_spacing
            y = start_y + j * v_spacing
            layer_pos.append((x, y))
        positions.append(layer_pos)
    
    # Връзки между слоевете
    for i in range(n_layers - 1):
        for pos1 in positions[i]:
            for pos2 in positions[i + 1]:
                ax.plot([pos1[0], pos2[0]], [pos1[1], pos2[1]], 
                       'gray', alpha=0.3, linewidth=0.5)
    
    # Невроните
    colors = ['lightblue', 'lightgreen', 'lightgreen', 'coral']
    for i, layer_pos in enumerate(positions):
        color = colors[min(i, len(colors)-1)] if i < len(layer_sizes)-1 else colors[-1]
        for pos in layer_pos:
            circle = plt.Circle(pos, 0.3, color=color, ec='black', linewidth=2, zorder=4)
            ax.add_patch(circle)
    
    # Етикети
    if layer_names:
        for i, name in enumerate(layer_names):
            x = i * h_spacing
            ax.text(x, -1.2, name, ha='center', fontsize=11)
    
    ax.set_xlim(-1, (n_layers-1) * h_spacing + 1)
    ax.set_ylim(-2, max_neurons * v_spacing)
    ax.set_aspect('equal')
    ax.axis('off')

# Примерна архитектура: 4 → 5 → 3 → 2
fig, ax = plt.subplots(figsize=(12, 6))
layer_sizes = [4, 5, 3, 2]
layer_names = ['Вход\n(4 признака)', 'Скрит слой 1\n(5 неврона)', 
               'Скрит слой 2\n(3 неврона)', 'Изход\n(2 класа)']
draw_neural_network(ax, layer_sizes, layer_names)
ax.set_title('Feedforward невронна мрежа: 4 → 5 → 3 → 2', fontsize=14, pad=20)
plt.show()

print(f"📊 Брой параметри:")
print(f"   W¹: 4×5 = 20, b¹: 5 → 25 параметъра")
print(f"   W²: 5×3 = 15, b²: 3 → 18 параметъра")
print(f"   W³: 3×2 = 6,  b³: 2 → 8 параметъра")
print(f"   Общо: 51 параметъра")

---
## 3. Право разпространение на сигнала (Forward Pass)

### Как изчисляваме изхода?

**Формули:**

$$z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$$
$$a^{[l]} = f(z^{[l]})$$

Където $a^{[0]} = x$ (входът).

**Пример за 2-слойна мрежа:**
1. $z^{[1]} = W^{[1]} x + b^{[1]}$, $a^{[1]} = \text{ReLU}(z^{[1]})$
2. $z^{[2]} = W^{[2]} a^{[1]} + b^{[2]}$, $\hat{y} = \sigma(z^{[2]})$

In [ ]:
# Имплементация на forward pass
def relu(z):
    return np.maximum(0, z)

def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def forward_pass(x, weights, biases):
    """
    Forward pass за feedforward мрежа.
    
    Args:
        x: вход, shape (n_features,)
        weights: списък от тегла [W1, W2, ...]
        biases: списък от biases [b1, b2, ...]
    
    Returns:
        activations: списък от активации [a0, a1, ...]
        pre_activations: списък от z стойности [z1, z2, ...]
    """
    activations = [x]  # a[0] = x
    pre_activations = []
    
    n_layers = len(weights)
    
    for l in range(n_layers):
        z = np.dot(weights[l], activations[-1]) + biases[l]
        pre_activations.append(z)
        
        # Последният слой: sigmoid, иначе ReLU
        if l == n_layers - 1:
            a = sigmoid(z)
        else:
            a = relu(z)
        
        activations.append(a)
    
    return activations, pre_activations

print("✓ Forward pass имплементиран!")

In [ ]:
# Конкретен пример: мрежа 3 → 4 → 2 → 1
np.random.seed(42)

# Инициализираме тегла (малки случайни стойности)
W1 = np.random.randn(4, 3) * 0.5  # 3 входа → 4 неврона
b1 = np.zeros(4)
W2 = np.random.randn(2, 4) * 0.5  # 4 → 2
b2 = np.zeros(2)
W3 = np.random.randn(1, 2) * 0.5  # 2 → 1 (изход)
b3 = np.zeros(1)

weights = [W1, W2, W3]
biases = [b1, b2, b3]

# Примерен вход
x = np.array([0.5, -0.2, 0.8])

# Forward pass
activations, pre_activations = forward_pass(x, weights, biases)

print("📊 Forward Pass стъпка по стъпка:\n")
print(f"Вход x = {x}")
print(f"Размерност: {x.shape}\n")

for l in range(len(weights)):
    print(f"--- Слой {l+1} ---")
    print(f"W{l+1} shape: {weights[l].shape}")
    print(f"z{l+1} = W{l+1} · a{l} + b{l+1}")
    print(f"z{l+1} = {pre_activations[l].round(3)}")
    act_name = 'sigmoid' if l == len(weights)-1 else 'ReLU'
    print(f"a{l+1} = {act_name}(z{l+1}) = {activations[l+1].round(3)}")
    print()

print(f"🎯 Изход ŷ = {activations[-1][0]:.4f}")

In [ ]:
# Визуализация на активациите
fig, axes = plt.subplots(1, 4, figsize=(14, 4))

layer_names = ['Вход x', 'Слой 1 (ReLU)', 'Слой 2 (ReLU)', 'Изход (Sigmoid)']

for i, (ax, a, name) in enumerate(zip(axes, activations, layer_names)):
    bars = ax.bar(range(len(a)), a, color='steelblue', edgecolor='black')
    ax.set_title(name)
    ax.set_xlabel('Неврон')
    if i == 0:
        ax.set_ylabel('Стойност')
    
    # Показваме стойностите
    for bar, val in zip(bars, a):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
               f'{val:.2f}', ha='center', fontsize=9)
    
    ax.set_ylim(min(a.min(), 0) - 0.1, max(a.max(), 0) + 0.3)

plt.tight_layout()
plt.show()

print("💡 Забележете: ReLU занулява отрицателните стойности (видимо в Слой 1 и 2)")

### Batch обработка: Ефективност чрез матрици

Вместо да обработваме примерите един по един, използваме **матрични операции**.

**Един пример:** $z = Wx + b$

**Batch от m примера:** $Z = WX^T + b$ (broadcasting)

Където $X$ е матрица $(m, n)$ — m примера с n признака.

In [ ]:
# Batch forward pass
def forward_pass_batch(X, weights, biases):
    """
    Forward pass за batch от примери.
    X: shape (m, n_features)
    """
    m = X.shape[0]
    activations = [X]  # (m, n_features)
    pre_activations = []
    
    n_layers = len(weights)
    
    for l in range(n_layers):
        # Z = X @ W.T + b, shape: (m, n_neurons)
        Z = np.dot(activations[-1], weights[l].T) + biases[l]
        pre_activations.append(Z)
        
        if l == n_layers - 1:
            A = sigmoid(Z)
        else:
            A = relu(Z)
        
        activations.append(A)
    
    return activations, pre_activations

# Batch от 5 примера
X_batch = np.random.randn(5, 3)  # 5 примера, 3 признака

activations_batch, _ = forward_pass_batch(X_batch, weights, biases)

print("📊 Batch Forward Pass:\n")
print(f"Вход X shape: {X_batch.shape}")
print(f"Изход ŷ shape: {activations_batch[-1].shape}")
print(f"\nПредсказания за 5-те примера:")
print(activations_batch[-1].flatten().round(4))

---
## 4. Loss функции: Какво оптимизираме?

### Цел на обучението

**Искаме:** Модел, който минимизира грешката върху данните.

**Формално:** $\min_\theta \mathcal{L}(\theta)$

Където $\theta = \{W^{[1]}, b^{[1]}, W^{[2]}, b^{[2]}, ...\}$ са всички параметри.

### Binary Cross-Entropy (BCE)

За **бинарна класификация** (spam/не-spam, положително/отрицателно):

$$\mathcal{L} = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{y}^{(i)}) + (1-y^{(i)}) \log(1-\hat{y}^{(i)}) \right]$$

**Интуиция:**
- Ако $y=1$: искаме $\hat{y} \to 1$, $\log(\hat{y}) \to 0$
- Ако $y=0$: искаме $\hat{y} \to 0$, $\log(1-\hat{y}) \to 0$

In [ ]:
# Binary Cross-Entropy Loss
def binary_cross_entropy(y_true, y_pred, epsilon=1e-15):
    """
    BCE loss.
    epsilon предотвратява log(0).
    """
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return loss

# Примери
print("📊 Binary Cross-Entropy Loss:\n")

examples = [
    (1, 0.9, "Вярно positive, висока увереност"),
    (1, 0.5, "Вярно positive, несигурно"),
    (1, 0.1, "Вярно positive, грешка!"),
    (0, 0.1, "Вярно negative, висока увереност"),
    (0, 0.9, "Вярно negative, грешка!"),
]

print(f"{'y_true':>8} {'ŷ':>8} {'Loss':>10} Описание")
print("-" * 60)

for y_true, y_pred, desc in examples:
    loss = binary_cross_entropy(np.array([y_true]), np.array([y_pred]))
    print(f"{y_true:>8} {y_pred:>8.1f} {loss:>10.4f} {desc}")

print("\n💡 Loss е нисък при правилни, уверени предсказания.")

In [ ]:
# Визуализация на BCE loss
y_pred_range = np.linspace(0.01, 0.99, 100)

# Loss за y=1 и y=0
loss_y1 = -np.log(y_pred_range)  # когато y=1
loss_y0 = -np.log(1 - y_pred_range)  # когато y=0

plt.figure(figsize=(10, 6))
plt.plot(y_pred_range, loss_y1, label='y = 1 (positive)', linewidth=2)
plt.plot(y_pred_range, loss_y0, label='y = 0 (negative)', linewidth=2)

# Анотации
plt.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
plt.annotate('Несигурност', xy=(0.5, 0.7), fontsize=10)

plt.xlabel('Предсказание ŷ', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Binary Cross-Entropy: наказва грешни уверени предсказания', fontsize=14)
plt.legend(fontsize=11)
plt.ylim(0, 5)
plt.grid(True, alpha=0.3)
plt.show()

print("💡 При y=1: loss → ∞ когато ŷ → 0 (уверена грешка)")
print("   При y=0: loss → ∞ когато ŷ → 1 (уверена грешка)")

### Softmax и Categorical Cross-Entropy

За **многокласова класификация**:

**Softmax:** Превръща logits в вероятности
$$\hat{y}_j = \frac{e^{z_j}}{\sum_{k=1}^{K} e^{z_k}}$$

**Cross-Entropy Loss:**
$$\mathcal{L} = -\frac{1}{m} \sum_{i=1}^{m} \sum_{j=1}^{K} y_j^{(i)} \log(\hat{y}_j^{(i)})$$

**Важно за LLM:** Next-token prediction използва точно тази loss!

In [ ]:
# Softmax функция
def softmax(z):
    """Numerically stable softmax."""
    z_shifted = z - np.max(z, axis=-1, keepdims=True)
    exp_z = np.exp(z_shifted)
    return exp_z / np.sum(exp_z, axis=-1, keepdims=True)

# Пример: класификация на тема (4 класа)
classes = ['sports', 'politics', 'tech', 'science']
logits = np.array([2.0, 1.0, 0.5, -1.0])  # "суров" изход от мрежата

probs = softmax(logits)

print("📊 Softmax трансформация:\n")
print(f"{'Клас':<12} {'Logit z':>10} {'P(клас)':>10}")
print("-" * 35)
for cls, z, p in zip(classes, logits, probs):
    bar = '█' * int(p * 20)
    print(f"{cls:<12} {z:>10.2f} {p:>10.3f} {bar}")

print(f"\n   Сума на вероятностите: {probs.sum():.4f}")

In [ ]:
# Температура в softmax (preview за generation в LLM)
def softmax_with_temp(z, temperature=1.0):
    return softmax(z / temperature)

logits = np.array([2.0, 1.0, 0.5, -1.0])
temperatures = [0.5, 1.0, 2.0, 5.0]

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(classes))
width = 0.2

for i, temp in enumerate(temperatures):
    probs = softmax_with_temp(logits, temp)
    ax.bar(x + i*width, probs, width, label=f'T={temp}')

ax.set_xticks(x + 1.5*width)
ax.set_xticklabels(classes)
ax.set_ylabel('Вероятност')
ax.set_title('Softmax с различни температури')
ax.legend()
plt.show()

print("💡 Температура контролира 'остротата' на разпределението:")
print("   • T < 1: по-уверени (по-остри) предсказания")
print("   • T > 1: по-равномерно разпределение")
print("   → Използва се при генериране на текст от LLM!")

---
## 5. Обратно разпространение на сигнала (Backpropagation)

### Идеята: Как да обновим теглата?

**Проблем:** Имаме loss $\mathcal{L}$. Как да намерим $\frac{\partial \mathcal{L}}{\partial W^{[l]}}$?

**Решение:** Chain rule (верижно правило)

$$\frac{\partial \mathcal{L}}{\partial W^{[1]}} = \frac{\partial \mathcal{L}}{\partial a^{[L]}} \cdot \frac{\partial a^{[L]}}{\partial z^{[L]}} \cdot \frac{\partial z^{[L]}}{\partial a^{[L-1]}} \cdots \frac{\partial z^{[1]}}{\partial W^{[1]}}$$

### Интуиция: Градиентът "тече" назад

```
Forward:  x → [W¹] → a¹ → [W²] → a² → [W³] → ŷ → L
                                                 ↓
Backward: ∂L/∂W¹ ← δ¹ ← [W²ᵀ] ← δ² ← [W³ᵀ] ← δ³ ← ∂L/∂ŷ
```

Където $\delta^{[l]} = \frac{\partial \mathcal{L}}{\partial z^{[l]}}$ е "грешката" на слой $l$.

In [ ]:
# Производни на активационните функции
def relu_derivative(z):
    return (z > 0).astype(float)

def sigmoid_derivative(a):
    """Производна на sigmoid: σ'(z) = σ(z)(1-σ(z)) = a(1-a)"""
    return a * (1 - a)

# Демонстрация
z = np.array([-2, -1, 0, 1, 2])
a_sig = sigmoid(z)

print("📊 Производни:\n")
print(f"{'z':>6} {'ReLU(z)':>10} {'ReLU\'(z)':>10} {'σ(z)':>10} {'σ\'(z)':>10}")
print("-" * 50)
for i in range(len(z)):
    print(f"{z[i]:>6} {relu(z)[i]:>10.2f} {relu_derivative(z)[i]:>10.2f} "
          f"{a_sig[i]:>10.4f} {sigmoid_derivative(a_sig)[i]:>10.4f}")

In [ ]:
# Пълна имплементация на backpropagation
def backward_pass(y_true, activations, pre_activations, weights):
    """
    Backpropagation за feedforward мрежа с BCE loss.
    
    Returns:
        dW: градиенти за теглата
        db: градиенти за bias-ите
    """
    m = y_true.shape[0]  # batch size
    n_layers = len(weights)
    
    dW = [None] * n_layers
    db = [None] * n_layers
    
    # Изходен слой: ∂L/∂z = ŷ - y (за sigmoid + BCE)
    y_pred = activations[-1]
    delta = y_pred - y_true.reshape(-1, 1)  # (m, 1)
    
    # Backprop през слоевете (от последен към първи)
    for l in range(n_layers - 1, -1, -1):
        a_prev = activations[l]  # (m, n_prev)
        
        # Градиенти за параметрите
        dW[l] = np.dot(delta.T, a_prev) / m  # (n_current, n_prev)
        db[l] = np.mean(delta, axis=0)  # (n_current,)
        
        # Пропагираме грешката назад (ако не сме на първия слой)
        if l > 0:
            delta = np.dot(delta, weights[l]) * relu_derivative(pre_activations[l-1])
    
    return dW, db

print("✓ Backpropagation имплементиран!")

In [ ]:
# Gradient checking: Проверяваме дали backprop е правилен
def numerical_gradient(X, y, weights, biases, layer_idx, param_type, i, j, epsilon=1e-5):
    """Изчислява градиент числено (за проверка)."""
    if param_type == 'W':
        params = weights
    else:
        params = biases
    
    # Запазваме оригиналната стойност
    original = params[layer_idx].flat[i * params[layer_idx].shape[-1] + j] if param_type == 'W' \
               else params[layer_idx][i]
    
    # f(θ + ε)
    if param_type == 'W':
        params[layer_idx].flat[i * params[layer_idx].shape[-1] + j] = original + epsilon
    else:
        params[layer_idx][i] = original + epsilon
    activations_plus, _ = forward_pass_batch(X, weights, biases)
    loss_plus = binary_cross_entropy(y, activations_plus[-1].flatten())
    
    # f(θ - ε)
    if param_type == 'W':
        params[layer_idx].flat[i * params[layer_idx].shape[-1] + j] = original - epsilon
    else:
        params[layer_idx][i] = original - epsilon
    activations_minus, _ = forward_pass_batch(X, weights, biases)
    loss_minus = binary_cross_entropy(y, activations_minus[-1].flatten())
    
    # Възстановяваме
    if param_type == 'W':
        params[layer_idx].flat[i * params[layer_idx].shape[-1] + j] = original
    else:
        params[layer_idx][i] = original
    
    return (loss_plus - loss_minus) / (2 * epsilon)

# Тест
np.random.seed(42)
X_test = np.random.randn(10, 3)
y_test = np.random.randint(0, 2, 10)

# Forward и backward
activations, pre_activations = forward_pass_batch(X_test, weights, biases)
dW, db = backward_pass(y_test, activations, pre_activations, weights)

print("✓ Gradient Checking:\n")
print(f"{'Параметър':>15} {'Backprop':>12} {'Численo':>12} {'Разлика':>12}")
print("-" * 55)

# Проверяваме няколко градиента
for l in range(len(weights)):
    bp_grad = dW[l][0, 0]
    num_grad = numerical_gradient(X_test, y_test, weights, biases, l, 'W', 0, 0)
    diff = abs(bp_grad - num_grad)
    status = '✓' if diff < 1e-5 else '✗'
    print(f"W[{l}][0,0] {status:>8} {bp_grad:>12.6f} {num_grad:>12.6f} {diff:>12.2e}")

print("\n💡 Малките разлики (< 1e-5) потвърждават, че backprop е правилен!")

---
## 6. Оптимизационни алгоритми

### Gradient Descent: Основната идея

**Правило за обновяване:**
$$\theta := \theta - \alpha \nabla_\theta \mathcal{L}$$

Където $\alpha$ е learning rate.

**Проблеми:**
- Learning rate: твърде малък → бавно, твърде голям → дивергира
- Noisy градиенти при малки batch-ове
- Може да заседне в локални минимуми

In [ ]:
# Stochastic Gradient Descent
def sgd_update(params, grads, learning_rate):
    """SGD: θ = θ - α∇L"""
    updated = []
    for p, g in zip(params, grads):
        updated.append(p - learning_rate * g)
    return updated

print("✓ SGD имплементиран!")

### SGD с Momentum

**Идея:** Добавяме "инерция" — градиентът се натрупва.

$$v := \beta v + (1-\beta) \nabla_\theta \mathcal{L}$$
$$\theta := \theta - \alpha v$$

**Предимства:**
- Изглажда шумни градиенти
- Ускорява конвергенцията в "плоски" посоки
- Помага да се избягат плитки локални минимуми

In [ ]:
# SGD с Momentum
class MomentumOptimizer:
    def __init__(self, learning_rate=0.01, beta=0.9):
        self.lr = learning_rate
        self.beta = beta
        self.velocities = None
    
    def update(self, params, grads):
        if self.velocities is None:
            self.velocities = [np.zeros_like(p) for p in params]
        
        updated = []
        for i, (p, g) in enumerate(zip(params, grads)):
            self.velocities[i] = self.beta * self.velocities[i] + (1 - self.beta) * g
            updated.append(p - self.lr * self.velocities[i])
        
        return updated

print("✓ Momentum оптимизатор имплементиран!")

### Adam: Adaptive Moment Estimation

**Комбинира:** Momentum + адаптивен learning rate за всеки параметър.

$$m := \beta_1 m + (1-\beta_1) \nabla_\theta \mathcal{L}$$ (първи момент)
$$v := \beta_2 v + (1-\beta_2) (\nabla_\theta \mathcal{L})^2$$ (втори момент)
$$\hat{m} = \frac{m}{1-\beta_1^t}, \quad \hat{v} = \frac{v}{1-\beta_2^t}$$ (bias корекция)
$$\theta := \theta - \alpha \frac{\hat{m}}{\sqrt{\hat{v}} + \epsilon}$$

**Типични стойности:** $\beta_1 = 0.9$, $\beta_2 = 0.999$, $\alpha = 0.001$

In [ ]:
# Adam Optimizer
class AdamOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None  # First moment
        self.v = None  # Second moment
        self.t = 0     # Time step
    
    def update(self, params, grads):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]
        
        self.t += 1
        updated = []
        
        for i, (p, g) in enumerate(zip(params, grads)):
            # Update moments
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * g
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (g ** 2)
            
            # Bias correction
            m_hat = self.m[i] / (1 - self.beta1 ** self.t)
            v_hat = self.v[i] / (1 - self.beta2 ** self.t)
            
            # Update parameters
            updated.append(p - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon))
        
        return updated

print("✓ Adam оптимизатор имплементиран!")

In [ ]:
# Визуализация: Сравнение на оптимизатори на 2D функция
def rosenbrock(x, y):
    """Rosenbrock функция - трудна за оптимизация."""
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(x, y):
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])

# Стартова точка
start = np.array([-1.0, 1.0])

# Траектории за различни оптимизатори
def optimize_trajectory(start, optimizer_name, n_steps=200, lr=0.001):
    pos = start.copy()
    trajectory = [pos.copy()]
    
    if optimizer_name == 'SGD':
        for _ in range(n_steps):
            grad = rosenbrock_grad(pos[0], pos[1])
            pos = pos - lr * grad
            trajectory.append(pos.copy())
    
    elif optimizer_name == 'Momentum':
        v = np.zeros(2)
        beta = 0.9
        for _ in range(n_steps):
            grad = rosenbrock_grad(pos[0], pos[1])
            v = beta * v + (1 - beta) * grad
            pos = pos - lr * v
            trajectory.append(pos.copy())
    
    elif optimizer_name == 'Adam':
        m = np.zeros(2)
        v = np.zeros(2)
        beta1, beta2, eps = 0.9, 0.999, 1e-8
        for t in range(1, n_steps + 1):
            grad = rosenbrock_grad(pos[0], pos[1])
            m = beta1 * m + (1 - beta1) * grad
            v = beta2 * v + (1 - beta2) * (grad ** 2)
            m_hat = m / (1 - beta1 ** t)
            v_hat = v / (1 - beta2 ** t)
            pos = pos - lr * m_hat / (np.sqrt(v_hat) + eps)
            trajectory.append(pos.copy())
    
    return np.array(trajectory)

# Изчисляваме траектории
traj_sgd = optimize_trajectory(start, 'SGD', lr=0.0001)
traj_momentum = optimize_trajectory(start, 'Momentum', lr=0.0001)
traj_adam = optimize_trajectory(start, 'Adam', lr=0.01)

# Визуализация
fig, ax = plt.subplots(figsize=(12, 8))

# Контурна графика на функцията
x_range = np.linspace(-2, 2, 100)
y_range = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = rosenbrock(X, Y)

ax.contour(X, Y, Z, levels=np.logspace(-1, 3, 20), cmap='viridis', alpha=0.7)

# Траектории
ax.plot(traj_sgd[:, 0], traj_sgd[:, 1], 'r.-', label='SGD', alpha=0.8, markersize=3)
ax.plot(traj_momentum[:, 0], traj_momentum[:, 1], 'b.-', label='Momentum', alpha=0.8, markersize=3)
ax.plot(traj_adam[:, 0], traj_adam[:, 1], 'g.-', label='Adam', alpha=0.8, markersize=3)

# Маркери
ax.plot(start[0], start[1], 'ko', markersize=10, label='Start')
ax.plot(1, 1, 'r*', markersize=15, label='Optimum')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Сравнение на оптимизатори (Rosenbrock функция)')
ax.legend(loc='upper left')
plt.show()

print("💡 Adam конвергира най-бързо към оптимума (1, 1).")
print("   SGD е най-бавен, Momentum е междинен.")

### Кой оптимизатор да използвам?

| Оптимизатор | Предимства | Недостатъци | Кога? |
|-------------|------------|-------------|-------|
| **SGD** | Прост, добра генерализация | Бавен, чувствителен към LR | Изследвания, някои CV задачи |
| **SGD+Momentum** | По-бърз от SGD | Допълнителен хиперпараметър | Computer vision |
| **Adam** | Бърз, адаптивен, работи "out of the box" | По-голяма памет | **Препоръчителен за NLP/LLM** |
| **AdamW** | Adam + правилен weight decay | - | **GPT, BERT, LLaMA** |

---
## 7. Практика на обучението

### Пълен training loop

In [ ]:
# Пълен training loop
def train_network(X_train, y_train, X_val, y_val, layer_sizes, 
                  epochs=100, batch_size=32, learning_rate=0.001,
                  optimizer='adam', verbose=True):
    """
    Обучава feedforward мрежа.
    
    Args:
        layer_sizes: списък [n_input, n_hidden1, ..., n_output]
    
    Returns:
        weights, biases, history
    """
    n_samples = X_train.shape[0]
    n_layers = len(layer_sizes) - 1
    
    # Инициализация на тегла (Xavier)
    weights = []
    biases = []
    for l in range(n_layers):
        n_in, n_out = layer_sizes[l], layer_sizes[l+1]
        W = np.random.randn(n_out, n_in) * np.sqrt(2.0 / n_in)
        b = np.zeros(n_out)
        weights.append(W)
        biases.append(b)
    
    # Optimizer
    if optimizer == 'adam':
        opt_W = AdamOptimizer(learning_rate)
        opt_b = AdamOptimizer(learning_rate)
    else:
        opt_W = MomentumOptimizer(learning_rate)
        opt_b = MomentumOptimizer(learning_rate)
    
    # История
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(epochs):
        # Shuffle
        indices = np.random.permutation(n_samples)
        X_shuffled = X_train[indices]
        y_shuffled = y_train[indices]
        
        # Mini-batch training
        for i in range(0, n_samples, batch_size):
            X_batch = X_shuffled[i:i+batch_size]
            y_batch = y_shuffled[i:i+batch_size]
            
            # Forward
            activations, pre_activations = forward_pass_batch(X_batch, weights, biases)
            
            # Backward
            dW, db = backward_pass(y_batch, activations, pre_activations, weights)
            
            # Update
            weights = opt_W.update(weights, dW)
            biases = opt_b.update(biases, db)
        
        # Оценка след всяка епоха
        train_act, _ = forward_pass_batch(X_train, weights, biases)
        val_act, _ = forward_pass_batch(X_val, weights, biases)
        
        train_loss = binary_cross_entropy(y_train, train_act[-1].flatten())
        val_loss = binary_cross_entropy(y_val, val_act[-1].flatten())
        
        train_pred = (train_act[-1].flatten() > 0.5).astype(int)
        val_pred = (val_act[-1].flatten() > 0.5).astype(int)
        train_acc = np.mean(train_pred == y_train)
        val_acc = np.mean(val_pred == y_val)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        if verbose and (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:3d}: train_loss={train_loss:.4f}, val_loss={val_loss:.4f}, "
                  f"train_acc={train_acc:.3f}, val_acc={val_acc:.3f}")
    
    return weights, biases, history

print("✓ Training loop имплементиран!")

In [ ]:
# Създаваме синтетични данни за демонстрация
from sklearn.datasets import make_moons

X_synth, y_synth = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(
    X_synth, y_synth, test_size=0.2, random_state=42
)

print(f"📊 Синтетични данни (two moons):")
print(f"   Train: {X_train_s.shape[0]} примера")
print(f"   Val: {X_val_s.shape[0]} примера")
print(f"   Признаци: {X_train_s.shape[1]}")

# Визуализация
plt.figure(figsize=(8, 6))
plt.scatter(X_synth[y_synth==0, 0], X_synth[y_synth==0, 1], alpha=0.6, label='Клас 0')
plt.scatter(X_synth[y_synth==1, 0], X_synth[y_synth==1, 1], alpha=0.6, label='Клас 1')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Two Moons Dataset (нелинейно разделим)')
plt.legend()
plt.show()

In [ ]:
# Обучаваме невронна мрежа
print("🚀 Обучение на невронна мрежа (2 → 16 → 8 → 1):\n")

weights_trained, biases_trained, history = train_network(
    X_train_s, y_train_s, X_val_s, y_val_s,
    layer_sizes=[2, 16, 8, 1],
    epochs=100,
    batch_size=32,
    learning_rate=0.01,
    optimizer='adam'
)

print(f"\n✓ Финални резултати:")
print(f"   Train accuracy: {history['train_acc'][-1]:.3f}")
print(f"   Val accuracy: {history['val_acc'][-1]:.3f}")

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training и Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training и Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Наблюдения:")
print("   • Train и Val loss намаляват заедно → добро обучение")
print("   • Кривите са близки → няма силно пренагаждане")

In [ ]:
# Визуализация на decision boundary
def plot_decision_boundary(X, y, weights, biases, title='Decision Boundary'):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    X_grid = np.c_[xx.ravel(), yy.ravel()]
    activations, _ = forward_pass_batch(X_grid, weights, biases)
    Z = activations[-1].reshape(xx.shape)
    
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, levels=50, cmap='RdBu', alpha=0.8)
    plt.colorbar(label='P(class=1)')
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    
    plt.scatter(X[y==0, 0], X[y==0, 1], c='red', edgecolors='black', s=50, label='Клас 0')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='blue', edgecolors='black', s=50, label='Клас 1')
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title(title)
    plt.legend()
    plt.show()

plot_decision_boundary(X_synth, y_synth, weights_trained, biases_trained,
                       'Невронна мрежа: Нелинейна граница на решение')

print("💡 Невронната мрежа създава нелинейна граница, която разделя двете 'луни'!")

### Пренагаждане и регуляризация

**Симптоми на пренагаждане:**
- Train loss намалява, Val loss расте
- Train accuracy >> Val accuracy

**Решения:**
- **Dropout:** Изключва случайни неврони по време на обучение
- **Weight decay (L2):** Добавя $\lambda ||\theta||^2$ към loss
- **Early stopping:** Спира, когато val loss спре да намалява

In [ ]:
# Демонстрация на пренагаждане с малък dataset
# Взимаме само 100 примера
X_small = X_train_s[:100]
y_small = y_train_s[:100]

print("📊 Демонстрация на пренагаждане (100 примера, голяма мрежа):\n")

# Голяма мрежа за малък dataset
weights_overfit, biases_overfit, history_overfit = train_network(
    X_small, y_small, X_val_s, y_val_s,
    layer_sizes=[2, 64, 32, 16, 1],  # Много параметри за 100 примера
    epochs=200,
    batch_size=16,
    learning_rate=0.01,
    verbose=False
)

# Визуализация
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(history_overfit['train_loss'], label='Train Loss', linewidth=2)
ax.plot(history_overfit['val_loss'], label='Val Loss', linewidth=2)
ax.axvline(x=np.argmin(history_overfit['val_loss']), color='red', linestyle='--', 
           label='Оптимална точка', alpha=0.7)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Пренагаждане: Train loss намалява, Val loss расте')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

best_epoch = np.argmin(history_overfit['val_loss'])
print(f"💡 Пренагаждане започва около epoch {best_epoch}")
print(f"   Train acc: {history_overfit['train_acc'][-1]:.3f}, Val acc: {history_overfit['val_acc'][-1]:.3f}")
print(f"   → Early stopping би спряло на epoch {best_epoch}")

---
## 8. Невронни мрежи за текстова класификация

### От текст до невронна мрежа

```
Текст → Токенизация → Embeddings → Pooling → Dense layers → Изход
                      (Лекция 2)            (днес)
```

**Pooling стратегии:**
- **Mean pooling:** Средно на всички token embeddings
- **Max pooling:** Максимум по всяко измерение
- **[CLS] token:** Специален token за класификация (BERT)

In [ ]:
# Зареждаме 20 Newsgroups (бинарна задача за простота)
categories = ['sci.space', 'rec.sport.baseball']

newsgroups = fetch_20newsgroups(
    subset='all',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    random_state=42
)

texts = newsgroups.data
labels = newsgroups.target

# Разделяме
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

X_train_text, X_val_text, y_train_text, y_val_text = train_test_split(
    X_train_text, y_train_text, test_size=0.2, random_state=42, stratify=y_train_text
)

print(f"📚 20 Newsgroups (binary classification):")
print(f"   Категории: {categories}")
print(f"   Train: {len(X_train_text)}, Val: {len(X_val_text)}, Test: {len(X_test_text)}")

In [ ]:
# TF-IDF векторизация
vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')

X_train_vec = vectorizer.fit_transform(X_train_text).toarray()
X_val_vec = vectorizer.transform(X_val_text).toarray()
X_test_vec = vectorizer.transform(X_test_text).toarray()

print(f"📊 TF-IDF векторизация:")
print(f"   Размерност: {X_train_vec.shape[1]} признака")
print(f"   Train shape: {X_train_vec.shape}")

In [ ]:
# Baseline: Logistic Regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_vec, y_train_text)

logreg_train_acc = logreg.score(X_train_vec, y_train_text)
logreg_val_acc = logreg.score(X_val_vec, y_val_text)

print(f"📊 Baseline (Logistic Regression):")
print(f"   Train accuracy: {logreg_train_acc:.3f}")
print(f"   Val accuracy: {logreg_val_acc:.3f}")

In [ ]:
# Невронна мрежа за текстова класификация
print("🚀 Обучение на невронна мрежа за текстова класификация:\n")
print(f"   Архитектура: {X_train_vec.shape[1]} → 256 → 64 → 1\n")

weights_text, biases_text, history_text = train_network(
    X_train_vec, y_train_text, X_val_vec, y_val_text,
    layer_sizes=[X_train_vec.shape[1], 256, 64, 1],
    epochs=50,
    batch_size=32,
    learning_rate=0.001,
    optimizer='adam'
)

print(f"\n📊 Сравнение:")
print(f"   Logistic Regression val acc: {logreg_val_acc:.3f}")
print(f"   Neural Network val acc:      {history_text['val_acc'][-1]:.3f}")

In [ ]:
# Финална оценка на тестовия набор
test_act, _ = forward_pass_batch(X_test_vec, weights_text, biases_text)
y_test_pred = (test_act[-1].flatten() > 0.5).astype(int)

nn_test_acc = np.mean(y_test_pred == y_test_text)
logreg_test_acc = logreg.score(X_test_vec, y_test_text)

print("📊 Финална оценка на TEST набор:\n")
print(f"   Logistic Regression: {logreg_test_acc:.3f}")
print(f"   Neural Network:      {nn_test_acc:.3f}")

print("\n" + "="*50)
print("Neural Network Classification Report:")
print("="*50)
print(classification_report(y_test_text, y_test_pred, target_names=categories))

### Ограничения на този подход

| Ограничение | Проблем | Решение |
|-------------|---------|--------|
| **Bag-of-words** | Губи ред на думите | RNN, Transformers (Лекция 5) |
| **Фиксиран речник** | Нови думи = OOV | Subword tokenization (Лекция 3) |
| **Без контекст** | "bank" = ? | Контекстуални embeddings (BERT) |
| **Pooling** | Средното губи информация | Attention механизъм |

---
## 9. Обобщение и мост към Трансформаторите

### Ключови изводи от днес

**1. Невронните мрежи надграждат линейните модели**
- Нелинейни активации позволяват сложни граници на решение
- Автоматично научават представяния (feature learning)

**2. Forward pass и Backpropagation**
- Forward: изчисляваме изхода слой по слой
- Backward: градиентите "текат" назад чрез chain rule

**3. Оптимизация**
- Adam е стандартът за NLP/LLM
- Learning rate е критичен хиперпараметър

**4. Training curves разкриват проблеми**
- Голяма разлика train/val → пренагаждане
- Regularization: dropout, weight decay, early stopping

### Следваща лекция: Трансформатори

**Въпроси, на които ще отговорим:**

- Как да обработим последователности, запазвайки реда?
- Какво е механизмът на вниманието (attention)?
- Как работи архитектурата на Transformer?
- Какво е BERT и как се различава от GPT?

**Защо е важно:**
- Transformers са основата на **всички** съвременни LLM
- GPT, Claude, LLaMA, Gemini — всички са Transformers

```
Feedforward NN (днес) → RNN → Attention → Transformer → GPT/BERT
                              ↑
                         Лекция 5
```

---
## Ресурси

### Препоръчително четене

**Статии:**
1. Rumelhart, Hinton, Williams (1986) - "Learning representations by back-propagating errors" (оригиналната backprop статия)
2. Kingma & Ba (2014) - "Adam: A Method for Stochastic Optimization"
3. Srivastava et al. (2014) - "Dropout: A Simple Way to Prevent Neural Networks from Overfitting"

**Учебници:**
1. "Deep Learning" - Goodfellow, Bengio, Courville, Глава 6 (Feedforward Networks)
2. "Neural Networks and Deep Learning" - Michael Nielsen (безплатен онлайн)

**Online:**
1. Stanford CS231n - Convolutional Neural Networks
2. 3Blue1Brown - Neural Networks video series (отлична визуална интуиция)
3. Andrej Karpathy - "A Recipe for Training Neural Networks" (блог пост)

### Упражнения за вкъщи

**Упражнение 1: Имплементация от нулата**
- Имплементирайте backprop за 3-слойна мрежа в NumPy
- Добавете L2 regularization
- Верифицирайте с gradient checking

**Упражнение 2: Сравнение на оптимизатори**
- Обучете един и същ модел с SGD, Momentum, Adam
- Начертайте training curves
- Експериментирайте с различни learning rates

**Упражнение 3: Диагностика на training**
- Дадени са training curves с различни проблеми
- Идентифицирайте: пренагаждане, ъндърфитване, лош learning rate
- Предложете решения

**Упражнение 4: Текстова класификация**
- Използвайте IMDB reviews dataset
- Сравнете TF-IDF + NN с Word2Vec embeddings + NN
- Анализирайте грешките

---
## Край на Лекция 4

### Благодаря за вниманието!

**Въпроси?**

---

**Следваща лекция:** Трансформатори (Attention, Self-Attention, BERT)